# GRU for Fake News Detection

This notebook demonstrates how to use **GRU (Gated Recurrent Unit)** networks for classifying news as **Fake** or **Real**.  
We use the same [Fake and Real News Dataset](https://www.kaggle.com/datasets/clmentbisaillon/fake-and-real-news-dataset) from Kaggle.

GRU is a type of RNN similar to LSTM but **simpler and faster**, with fewer gates:
- GRU has **Update** and **Reset gates** instead of 3 gates in LSTM.  
- Can handle **long-term dependencies** like LSTM, but with **fewer parameters**.

In [1]:
import pandas as pd

# Load Fake and Real news CSVs
fake_df = pd.read_csv('dataset/fake-news/Fake.csv')
true_df = pd.read_csv('dataset/fake-news/True.csv')

# Add label column
fake_df['label'] = 0
true_df['label'] = 1

# Combine and shuffle
df = pd.concat([fake_df, true_df], ignore_index=True)
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

# Keep only the text and label
df = df[['text', 'label']]

# Check
df.head()


,text,label
0,"21st Century Wire says Ben Stein, reputable pr...",0
1,WASHINGTON (Reuters) - U.S. President Donald T...,1
2,(Reuters) - Puerto Rico Governor Ricardo Rosse...,1
3,"On Monday, Donald Trump once again embarrassed...",0
4,"GLASGOW, Scotland (Reuters) - Most U.S. presid...",1


In [2]:
# Prepare Features and Labels
# Independent feature
X = df['text']

# Dependent feature
y = df['label']

# Check shape
print("Number of samples:", len(X))
print("Number of labels:", len(y))


Number of samples: 44898
Number of labels: 44898


In [3]:
# Text Preprocessing 
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Vocabulary size
voc_size = 5000

# One-hot encode text
onehot_repr = [one_hot(words, voc_size) for words in X]

# Pad sequences
sent_length = 20
embedded_docs = pad_sequences(onehot_repr, padding='post', maxlen=sent_length)

# Check first padded text
print(embedded_docs[0])


[2762 2198   27 4553 1893 1816 4294 4614 4261 4553 3133 1469  789 2744
 2968 4968 3305 2402 3478  800]


In [4]:
# Prepare Data for Model
import numpy as np
from sklearn.model_selection import train_test_split

X_final = np.array(embedded_docs)
y_final = np.array(y)

X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

print("Training data shape:", X_train.shape)
print("Testing data shape:", X_test.shape)


Training data shape: (30081, 20)
Testing data shape: (14817, 20)


In [5]:
# Build GRU Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GRU, Dense, Dropout

embedding_vector_features = 40

model = Sequential([
    Embedding(input_dim=voc_size, output_dim=embedding_vector_features, input_length=sent_length),
    Dropout(0.3),
    GRU(100),
    Dropout(0.3),
    Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Build model to initialize parameters
model.build(input_shape=(None, sent_length))
model.summary()


D:\miniconda_setup\Lib\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ (None, 20, 40)              │         200,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 20, 40)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru (GRU)                            │ (None, 100)                 │          42,600 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 100)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1)                   │             101 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 242,701 (948.05 KB)

 Trainable params: 242,701 (948.05 KB)

 Non-trainable params: 0 (0.00 B)

In [6]:
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=64)

Epoch 1/10
471/471 ━━━━━━━━━━━━━━━━━━━━ 26s 42ms/step - accuracy: 0.9234 - loss: 0.1959 - val_accuracy: 0.9426 - val_loss: 0.1670
Epoch 2/10
471/471 ━━━━━━━━━━━━━━━━━━━━ 19s 41ms/step - accuracy: 0.9610 - loss: 0.1028 - val_accuracy: 0.9565 - val_loss: 0.1180
Epoch 3/10
471/471 ━━━━━━━━━━━━━━━━━━━━ 19s 39ms/step - accuracy: 0.9711 - loss: 0.0795 - val_accuracy: 0.9559 - val_loss: 0.1201
Epoch 4/10
471/471 ━━━━━━━━━━━━━━━━━━━━ 19s 39ms/step - accuracy: 0.9774 - loss: 0.0617 - val_accuracy: 0.9595 - val_loss: 0.1229
Epoch 5/10
471/471 ━━━━━━━━━━━━━━━━━━━━ 19s 40ms/step - accuracy: 0.9834 - loss: 0.0478 - val_accuracy: 0.9591 - val_loss: 0.1376
Epoch 6/10
471/471 ━━━━━━━━━━━━━━━━━━━━ 18s 38ms/step - accuracy: 0.9866 - loss: 0.0389 - val_accuracy: 0.9584 - val_loss: 0.1468
Epoch 7/10
471/471 ━━━━━━━━━━━━━━━━━━━━ 18s 38ms/step - accuracy: 0.9896 - loss: 0.0324 - val_accuracy: 0.9585 - val_loss: 0.1510
Epoch 8/10
471/471 ━━━━━━━━━━━━━━━━━━━━ 18s 37ms/step - accuracy: 0.9903 - loss: 0.0277 - 

In [7]:
# Evaluate Model & Predictions
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
import numpy as np

# Predict
y_pred = model.predict(X_test)
y_pred = np.where(y_pred > 0.5, 1, 0)

# Metrics
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nAccuracy Score:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))


464/464 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step
Confusion Matrix:
 [[7496  258]
 [ 348 6715]]

Accuracy Score: 0.9591010325976919

Classification Report:
               precision    recall  f1-score   support

           0       0.96      0.97      0.96      7754
           1       0.96      0.95      0.96      7063

    accuracy                           0.96     14817
   macro avg       0.96      0.96      0.96     14817
weighted avg       0.96      0.96      0.96     14817



In [10]:
# Sample Prediction Visualization
import random

indices = random.sample(range(len(X_test)), 5)

for i in indices:
    text = X.iloc[i]
    true_label = y.iloc[i]
    
    pred_prob = model.predict(X_final[i].reshape(1, -1))[0][0]
    pred_label = 1 if pred_prob > 0.5 else 0
    
    print("="*80)
    print(f"📰 News: {text[:300]}...")
    print(f"✅ Actual: {'True' if true_label==1 else 'Fake'}")
    print(f" Predicted: {'True' if pred_label==1 else 'Fake'} (prob: {pred_prob:.2f})")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step
📰 News: Well, here s a Twitter post that backfired on Donald Trump.Over the weekend, billionaire Prince Alwaleed bin Talal, who owns a stake in Fox News, along with several other princes and former cabinet officials, were arrested for corruption by order of Crown Prince Mohammad bin Salman.Ironically, Trump...
✅ Actual: Fake
 Predicted: Fake (prob: 0.00)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step
📰 News: CHARLESTON, W. Va. (Reuters) - Don Blankenship, the former CEO of coal company Massey Energy who was recently released from jail after a sentence for violating mine safety laws, said on Wednesday he plans to run for U.S. Senate representing West Virginia. “It’s true,” he told Reuters in an email, wi...
✅ Actual: True
 Predicted: True (prob: 0.98)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step
📰 News: Legal Expert Jonathan Turley listened as Joe Scarborough laid out a lengthy timeline to make a case of obstruction on President Trump. His response is awesome and

GRU is simpler than LSTM, uses fewer parameters, and often trains faster.

Rest of the workflow (preprocessing, padding, embedding, train/test split) is identical to the LSTM notebook.